Fill in the following variables to start

In [3]:
HF_USR_NAME = 'shirwu'
TOOL_QA_ROOT = ''

### Upload to Huggingface

In [3]:
import pandas as pd
from datasets import Dataset
from datasets import DatasetDict

level = 'easy'
dataset = 'scirex'

dataset_dir = f'{dataset}-{level}.jsonl'
hf_dataset_name = f'toolqa_{dataset}_{level}'

df = pd.read_json(dataset_dir, lines=True)
df.head()

df['answer'] = df['answer'].apply(lambda x: str(x))
dataset = Dataset.from_pandas(df)

In [4]:
dataset_dict = DatasetDict({'train': dataset})
# push to hf for the ease for using dspy
# dataset_dict.push_to_hub(repo_id=hf_dataset_name, private=True)

## Setting Up

* ToolQA

Before loading our datasets and going to the execution part, we'll need to configure the `lm` in `dspy.settings`. For the purpose of this notebook we'll be using `gpt-4o`.

In [5]:
import os
import dspy
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)


dspy.settings.configure(
    lm=dspy.OpenAI(
        model="gpt-4o",
        api_key=os.getenv("OPENAI_API_KEY"),
        max_tokens=4000,
        temperature=0
    )
)

## Defining Signature

In [6]:
class ToolQASignature(dspy.Signature):
    """You will be given a question. Your task is to answer the question with a short response. 
    """
    
    question: str = dspy.InputField(
        prefix="Question:",
        desc="question to ask",
        format=lambda x: x.strip(),
    )
    answer: str = dspy.OutputField(
        prefix="Answer:",
        desc="answer to the question",
    )


## Loading Datasets

In [7]:
from random import sample
from dspy.datasets import DataLoader

dl = DataLoader()

In [8]:
tool_qa = dl.from_huggingface(
    f'{HF_USR_NAME}/' + hf_dataset_name,
    split="train",
    input_keys=("question", "answer"),
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [9]:
len(tool_qa)

100

In [10]:
import random
# set seed
random.seed(42)

train_idx = random.sample(range(len(tool_qa)), 40)
remaining_idx = list(set(range(len(tool_qa))) - set(train_idx))
test_idx = random.sample(remaining_idx, 60)

toolqa_train = [
    dspy.Example(question=example.question, answer=example.answer).with_inputs("question", "paper_id")
    for example in [tool_qa[i] for i in train_idx]
]
toolqa_test = [
    dspy.Example(question=example.question, answer=example.answer).with_inputs("question", "paper_id")
    for example in [tool_qa[i] for i in test_idx]
]

## Setting Up Tools

We'll setup `Avatar` modules for both signatures and all the `tools` can be used by each of the dataset. `Tool` is a pydantic model that Avatar expects the `tools` to be composed as more specifically it have 4 fields:

* `name` : Name of the tool
* `input_type` : Type of input the tool accepts
* `output_type` : Type of output the tool returns
* `tool` : The actual tool object

In [11]:
import os
import time
import uuid
import numpy as np
import jsonlines
from concurrent.futures import ProcessPoolExecutor
import sentence_transformers
import chromadb
from os import path as osp
from chromadb.config import Settings

EMBED_MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
CHROMA_PERSIST_DIRECTORY = osp.join(TOOL_QA_ROOT, "data/chroma_db/scirex-v2")
CHROMA_COLLECTION_NAME = "all"
CHROMA_SERVER_HOST = "localhost"
CHROMA_SERVER_HTTP_PORT = "8000"
FILE_PATH = osp.join(TOOL_QA_ROOT, "data/external_corpus/scirex/Preprocessed_Scirex.jsonl")

def sentence_embedding(model, texts):
    embeddings = model.encode(texts)
    return embeddings

def create_chroma_db(chroma_server_host, chroma_server_http_port, collection_name):
    chroma_client = chromadb.Client(Settings(
        chroma_api_impl="rest",
        chroma_server_host=chroma_server_host,
        chroma_server_http_port=chroma_server_http_port,
    ))
    collection = chroma_client.get_or_create_collection(name=collection_name)
    return collection

def create_chroma_db_local(persist_directory, collection_name):
    chroma_client = chromadb.PersistentClient(path=persist_directory)
    collection = chroma_client.get_or_create_collection(name=collection_name)
    return collection

def insert_to_db(texts, model_name, cuda_idx, db):
    model = sentence_transformers.SentenceTransformer(model_name, device=f"cuda:{cuda_idx}")

    batch_embeddings = []
    batch_texts = []
    start_time = time.time()
    print(f"Total Articles to process: {len(texts)}, Current Thread: {cuda_idx}.")
    for i, text in enumerate(texts):
        # 2. generate embedding
        embeddings = sentence_embedding(model, text).tolist()

        batch_embeddings.append(embeddings)
        batch_texts.append(text)
        # 3. add to vectorstore per 500 articles or last article
        if i % 100 == 0 or i == len(texts)-1:
            batch_ids = [str(uuid.uuid1()) for _ in batch_texts]
            db.add(
                embeddings=batch_embeddings,
                documents=batch_texts,
                ids = batch_ids
            )
            batch_embeddings = []
            batch_texts = []
            print(f"Completed Processing article count: {i}, Current Thread: {cuda_idx}, Time took: {time.time() - start_time}.")
    print(f"Thread {cuda_idx} Completed. Total time took for thread: {time.time() - start_time}.")


# Multi-processing
def query_llm(query, is_local=True, start=None, end=None):
    cuda_idxes = [0]
    number_of_processes = len(cuda_idxes)
    input_texts = []
    db = create_chroma_db_local(CHROMA_PERSIST_DIRECTORY, CHROMA_COLLECTION_NAME)
    with open(FILE_PATH, 'r') as f:
        for item in jsonlines.Reader(f):
            input_texts.append(item["content"])
    # input_texts = np.array_split(input_texts, number_of_processes)

    args = ((input_texts[i], EMBED_MODEL_NAME, cuda_idxes[i], is_local) for i in range(number_of_processes))

    # if there is no file under the directory "/localscratch/yzhuang43/ra-llm/retrieval_benchmark/data/chroma_db/agenda", insert the data into the db
    # You should run insert_to_db the first time!
    if len(os.listdir(CHROMA_PERSIST_DIRECTORY)) == 0:
        insert_to_db(input_texts, model_name=EMBED_MODEL_NAME, cuda_idx=0, db=db)

    input_paths = np.array_split(input_texts, number_of_processes)
    with ProcessPoolExecutor(number_of_processes) as executor:
        executor.map(insert_to_db, args)
    model = sentence_transformers.SentenceTransformer(EMBED_MODEL_NAME, device=f"cuda:0")
    query_embedding = sentence_embedding(model, query).tolist()
    results = db.query(query_embeddings=query_embedding, n_results=3)
    retrieval_content = [result for result in results['documents'][0]]
    # print(retrieval_content)
    retrieval_content = '\n'.join(retrieval_content)
    return retrieval_content

query = "What is an atom"
print(query_llm(query))

paragraph : Sentence Level For representing a document , one can split it up into sentences , with each memory slot encoding one sentence . Both the key and the value encode the entire sentence as a bag - of - words . As the key and value are the same in this case , this is identical to a standard MemNN and this approach has been used in several papers .
paragraph : Window Level Documents are split up into windows of words ; in our tasks we only include windows where the center word is an entity . Windows are represented using bag - of - words . Window representations for MemNNs have been shown to work well previously . However , in Key - Value MemNNs we encode the key as the entire window , and the value as only the center word , which is not possible in the MemNN architecture . This makes sense because the entire window is more likely to be pertinent as a match for the question ( as the key ) , whereas the entity at the center is more pertinent as a match for the answer ( as the valu

In [12]:
from dspy.predict.avatar import Tool, Avatar
from langchain_community.utilities import GoogleSerperAPIWrapper, ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.tools import BaseTool, StructuredTool, tool

def RETRIEVE(query: str) -> str:
    """If you want to search for some paper information, you can use this tool and input a natural language query. For example, RETRIEVE(\'Which method achieves the highest PCK score?\') returns relevant paper paragraph and meta data."""
    return query_llm(query)

tools = [
    Tool(
        tool=StructuredTool.from_function(RETRIEVE),
        name="RETRIEVE",
        desc="If you want to search for some paper information, you can use this tool and input a natural language query. For example, RETRIEVE('Which method achieves the highest PCK score?') returns relevant paper paragraph and meta data."
    ),
    Tool(
        tool=GoogleSerperAPIWrapper(),
        name="WEB_SEARCH",
        desc="If you have a question, you can use this tool to search the web for the answer."
    ),
    Tool(
        tool=ArxivAPIWrapper(),
        name="ARXIV_SEARCH",
        desc="Pass the arxiv paper id to get the paper information.",
        input_type="Arxiv Paper ID",
    )
]

Once we have defined our `tools`, we can now create an `Avatar` object by passing the `tools` and `signature`. It takes 2 more optional parameters `verbose` and `max_iters`. `verbose` is used to display the logs and `max_iters` is used to control the number of iterations in multi step execution. 

An avatar agent stops the tool usage iteration once it reaches `max_iters` or when it prompts `Finish`. You can also create custom tools too, all you need to make sure is:

* You pass is a class object.
* Implements `__init__` and `run` method.
* Must take 1 string a input and returns 1 string as output.

If your tool doesn't return or takes input a string then you can make a custom wrapper to take care of that for now. In future we'll try to enable a diverse tool usage.

In [13]:
actor_agent = Avatar(
    tools=tools,
    signature=ToolQASignature,
    verbose=False,
)

In [14]:
import time
import logging
from dataclasses import dataclass, field
from typing import Dict, List, Optional
from datetime import datetime
import tiktoken
from concurrent.futures import ThreadPoolExecutor, as_completed
import warnings
import copy
import tqdm
import logging
import warnings
import os

# Set up logging
# logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# Disable all INFO logging
logging.getLogger().setLevel(logging.WARNING)

# Silence all loggers that might be chatty
loggers_to_silence = [
    "httpx",
    "httpcore",
    "openai",
    "arxiv",
    "dspy",
    "langchain",
    "langchain_community",
    "requests",
    "urllib3",
    "tiktoken",
    "asyncio",
    "faiss",
    "anthropic"
]

for logger_name in loggers_to_silence:
    logging.getLogger(logger_name).setLevel(logging.WARNING)

# Suppress specific warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # Disable tokenizer parallelism warning

## Evaluation

Open enden QA tasks are hard to evaluate on rigid metrics like exact match. So, we'll be using an improvised LLM as Judge for the evaluation of our model on test set.

In [15]:
class Evaluator(dspy.Signature):
    """Please act as an impartial judge to evaluate whether the answer is correct based on the ground truth answer"""
    
    question: str = dspy.InputField(
        prefix="Question:",
        desc="question to ask",
    )
    reference_answer: str = dspy.InputField(
        prefix="Ground Truth Answer:",
        desc="Ground truth answer to the question.",
    )
    answer: str = dspy.InputField(
        prefix="Answer:",
        desc="Answer to the question given by the model.",
    )
    rationale: str = dspy.OutputField(
        prefix="Rationale:",
        desc="Explanation of why the answer is correct or incorrect.",
    )
    is_correct: float = dspy.OutputField(
        prefix="Correct:",
        desc="Whether the answer is correct. Give 0 if incorrect, 1 if correct, (0, 1) if partially correct.",
    )


evaluator = dspy.TypedPredictor(Evaluator)


def metric(example, prediction, trace=None):  
    # We found sometimes the ground truth answers are incomplete or the answer
    # is part of the ground truth answer. Therefore, for better comparison, 
    # we use a continuous value for the correct score   
    acc = float(
        evaluator(
            question=example.question,
            answer=prediction.answer,
            reference_answer=example.answer
        ).is_correct
    ) 
    print(prediction.answer, '|', example.answer, '=>', acc)
    return acc

print(toolqa_train[0])
metric(toolqa_train[0], prediction=dspy.Example(answer='physics'))

Example({'question': 'What is the corresponding Medium_Human-Normalized_Score score of the Ape-X method on Atari-57 dataset for Atari_Games task?', 'answer': '434.1%'}) (input_keys={'question', 'paper_id'})
physics | 434.1% => 0.0


0.0

For evaluation we can't use `dspy.Evaluate`, reason being that `Avatar` changes it's signature per iteration by adding the actions and it's results to it as fields. So we can create our own hacky thread safe evaluator for it.

In [16]:
import tqdm


os.environ["TOKENIZERS_PARALLELISM"] = "False"

from concurrent.futures import ThreadPoolExecutor

def process_example(example, signature):
    try:
        avatar = Avatar(
            signature,
            tools=tools,
            verbose=False,
        )
        prediction = avatar(**example.inputs().toDict())

        return metric(example, prediction)
    except Exception as e:
        print(e)
        return 0

# process_example(tool_qa[0], ToolQASignature)
def multi_thread_executor(test_set, signature, num_threads=60):
    total_score = 0
    total_examples = len(test_set)

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(process_example, example, signature) for example in test_set]

        for future in tqdm.tqdm(futures, total=total_examples, desc="Processing examples"):
            total_score += future.result()

    avg_metric = total_score / total_examples
    return avg_metric

def single_thread_executor(test_set, signature):
    total_score = 0
    total_examples = len(test_set)

    for example in tqdm.tqdm(test_set, desc="Processing examples"):
        total_score += process_example(example, signature)

    avg_metric = total_score / total_examples
    return avg_metric

In [17]:
import time
import logging
from dataclasses import dataclass, field
from typing import Dict, List, Optional
from datetime import datetime
import tiktoken
from concurrent.futures import ThreadPoolExecutor, as_completed
import warnings
import copy

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class APICallMetrics:
    timestamp: datetime
    tool_name: str
    tokens_in: int = 0
    tokens_out: int = 0
    execution_time: float = 0.0

@dataclass
class AvatarMetrics:
    total_calls: int = 0
    total_tokens_in: int = 0
    total_tokens_out: int = 0
    total_execution_time: float = 0.0
    calls_by_tool: Dict[str, int] = field(default_factory=dict)
    api_call_history: List[APICallMetrics] = field(default_factory=list)
    
    def add_call(self, metrics: APICallMetrics):
        self.total_calls += 1
        self.total_tokens_in += metrics.tokens_in
        self.total_tokens_out += metrics.tokens_out
        self.total_execution_time += metrics.execution_time
        self.calls_by_tool[metrics.tool_name] = self.calls_by_tool.get(metrics.tool_name, 0) + 1
        self.api_call_history.append(metrics)
    
    def merge(self, other: 'AvatarMetrics'):
        """Merge another AvatarMetrics instance into this one"""
        self.total_calls += other.total_calls
        self.total_tokens_in += other.total_tokens_in
        self.total_tokens_out += other.total_tokens_out
        self.total_execution_time += other.total_execution_time
        for tool, count in other.calls_by_tool.items():
            self.calls_by_tool[tool] = self.calls_by_tool.get(tool, 0) + count
        self.api_call_history.extend(other.api_call_history)

    def estimate_cost(self, model_name: str = "gpt-4") -> float:
        pricing = {
            "gpt-4": {"input": 2.5, "output": 10.0},
        }
        if model_name not in pricing:
            raise ValueError(f"Unknown model: {model_name}")
        
        rates = pricing[model_name]
        input_cost = (self.total_tokens_in / 1000000) * rates["input"]
        output_cost = (self.total_tokens_out / 1000000) * rates["output"]
        return input_cost + output_cost

class AvatarWithMetrics(Avatar):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.metrics = AvatarMetrics()
        self.tokenizer = tiktoken.encoding_for_model("gpt-4")
    
    def _count_tokens(self, text: str) -> int:
        try:
            return len(self.tokenizer.encode(str(text)))
        except Exception as e:
            logger.warning(f"Error counting tokens: {e}")
            return 0

    def _wrapped_tool_call(self, tool, input_text: str) -> str:
        start_time = time.time()
        tokens_in = self._count_tokens(input_text)
        
        try:
            result = tool.run(input_text)
        except Exception as e:
            logger.error(f"Tool execution error: {e}")
            raise
        finally:
            execution_time = time.time() - start_time
            tokens_out = self._count_tokens(str(result))
            
            metrics = APICallMetrics(
                timestamp=datetime.now(),
                tool_name=tool.name,
                tokens_in=tokens_in,
                tokens_out=tokens_out,
                execution_time=execution_time
            )
            self.metrics.add_call(metrics)
            
        return result

    def __call__(self, *args, **kwargs):
        start_time = time.time()
        result = super().__call__(*args, **kwargs)
        total_time = time.time() - start_time
        
        metrics = APICallMetrics(
            timestamp=datetime.now(),
            tool_name="main_llm",
            tokens_in=self._count_tokens(str(args) + str(kwargs)),
            tokens_out=self._count_tokens(str(result)),
            execution_time=total_time
        )
        self.metrics.add_call(metrics)
        
        return result

def multi_thread_executor(test_set, signature, num_threads=60):
    total_score = 0
    total_examples = len(test_set)
    combined_metrics = AvatarMetrics()

    start_time = time.time()
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []
        for example in test_set:
            def process_with_metrics(example=example):
                try:
                    avatar = AvatarWithMetrics(signature, tools=tools, verbose=False, max_iters=10)
                    prediction = avatar(**example.inputs().toDict())
                    return metric(example, prediction), avatar.metrics
                except Exception as e:
                    print(e)
                    return 0, AvatarMetrics()

            futures.append(executor.submit(process_with_metrics))

        for future in tqdm.tqdm(futures, total=total_examples, desc="Processing examples"):
            score, metrics = future.result()
            total_score += score
            # Only combine token counts and call counts, not execution times
            combined_metrics.total_calls += metrics.total_calls
            combined_metrics.total_tokens_in += metrics.total_tokens_in
            combined_metrics.total_tokens_out += metrics.total_tokens_out
            for tool, count in metrics.calls_by_tool.items():
                combined_metrics.calls_by_tool[tool] = combined_metrics.calls_by_tool.get(tool, 0) + count
            combined_metrics.api_call_history.extend(metrics.api_call_history)
    
    total_execution_time = time.time() - start_time
    combined_metrics.total_execution_time = total_execution_time

    avg_metric = total_score / total_examples
    return avg_metric, combined_metrics

def single_thread_executor(test_set, signature):
    total_score = 0
    total_examples = len(test_set)
    combined_metrics = AvatarMetrics()

    for example in tqdm.tqdm(test_set, desc="Processing examples"):
        try:
            avatar = AvatarWithMetrics(signature, tools=tools, verbose=False, max_iters=10)
            prediction = avatar(**example.inputs().toDict())
            score = metric(example, prediction)
            total_score += score
            # Combine metrics from this run
            for call in avatar.metrics.api_call_history:
                combined_metrics.add_call(call)
        except Exception as e:
            print(e)

    avg_metric = total_score / total_examples
    return avg_metric, combined_metrics

def format_metrics_report(metrics: AvatarMetrics, model_name: str = "gpt-4") -> str:
    cost = metrics.estimate_cost(model_name)
    
    report = f"""
Avatar Execution Metrics Report
==============================
Execution Time: {metrics.total_execution_time:.2f} seconds
Total API Calls: {metrics.total_calls}
Total Tokens: {metrics.total_tokens_in + metrics.total_tokens_out:,} ({metrics.total_tokens_in:,} in, {metrics.total_tokens_out:,} out)
Estimated Cost: ${cost:.4f}

Average Time per Call: {metrics.total_execution_time / metrics.total_calls:.2f} seconds

Tool Usage Breakdown:
-------------------
"""
    for tool, count in sorted(metrics.calls_by_tool.items()):
        report += f"{tool}: {count} calls\n"

    return report

In [18]:
score, metrics = multi_thread_executor(toolqa_test, ToolQASignature)

Processing examples:   2%|▏         | 1/60 [00:29<29:28, 29.97s/it]

The corresponding Accuracy score of the BB8 method on the LineMOD dataset for the 6D Pose Estimation task is 89.3%. | 83.9% => 0.0
DeepId2 method achieved an accuracy score of 99.15% on the Labeled Faces in the Wild dataset for the Face Verification task. | 99.15% => 1.0


Processing examples:   3%|▎         | 2/60 [00:36<15:44, 16.28s/it]

The Train Accuracy score of the 450D_DR-BiLSTM_Ensemble method on the SNLI dataset for the Natural Language Inference task is 94.8%. | 94.8 => 1.0
The corresponding Inception score of the CEGAN-Ent-VI method on the CIFAR-10 dataset for the Image Generation task is 7.07. | 7.07 => 1.0
The UAS score of the Arc-hybrid method on the Penn_Treebank dataset for the Dependency_Parsing task is 94.51%. | 93.56 => 0.0
The corresponding Percentage_error score of the Bi-LSTM___skip_connections_w__CTC method on the TIMIT dataset for the Speech Recognition task is 17.7%. | 17.7 => 1.0
The BLEU-2 score of the LeakGAN method on the Chinese Poems dataset for the Text Generation task is 0.456. | 0.881 => 0.0
The corresponding MAP score of the subCNN method on the PASCAL_VOC_2007 dataset for the Object Detection task is not explicitly mentioned in the retrieved results. However, the QSQS algorithm, which is an improvement over traditional methods, achieved a MAP score of 75.11% on the PASCAL VOC 2007 data

Processing examples:   7%|▋         | 4/60 [01:34<19:58, 21.40s/it]

...

The Frame__fps_ score of the CRF-RNN method on the Cityscapes dataset for Real-Time Semantic Segmentation task is not directly available from the search results. Further specific research or direct access to the original paper or dataset results may be required to obtain this information. | 1.4 => 0.0
The corresponding NLL_Test score of the Conv_DRAW method on the CIFAR-10 dataset for the Image_Generation task is -1791.15. | 3.58 => 0.0
The Unigram_Acc score of the ASR method on the SearchQA dataset for Open-Domain Question Answering task is not readily available from the current sources. Further specific research or access to detailed experimental results from relevant papers may be required to obtain this information. | 41.3 => 0.0
The corresponding AP score of the ACF-WIDER method on the WIDER_Face_Hard dataset for the Face Detection task is not directly available from the search results. Further specific research or access to the original paper or dataset results might be required 

Processing examples:  35%|███▌      | 21/60 [01:49<00:50,  1.30s/it]

The Percentage_correct score for the Deep Networks with Internal Selective Attention through Feedback Connections method on the CIFAR-100 dataset for the Image Classification task is not explicitly mentioned in the available resources. The paper mentions that dasNet outperforms the previous state-of-the-art model on unaugmented datasets, but specific percentage scores are not provided. | 66.2 => 0.0
The corresponding F1 score of the Yang et al. method on the CoNLL 2003 English dataset for the Named Entity Recognition (NER) task is not explicitly found in the retrieved documents. Further specific details about the Yang et al. method's performance on this dataset may require direct access to the original paper or additional specific resources. | 91.26 => 0.0
The Params score of the Past_Decode_Reg____AWD-LSTM-MoS___dyn__eval_ method on the Penn_Treebank__Word_Level_ dataset for the Language_Modelling task could not be found using the available tools. | 22M => 0.0
The Test_perplexity scor

Processing examples: 100%|██████████| 60/60 [01:54<00:00,  1.90s/it]

The Hits@1 score of the ComplEx method on the WN18 dataset for the Link Prediction task is approximately 0.089. | 0.936 => 0.0


In [19]:
print(f"Test Score: {score:.2f}")
print(format_metrics_report(metrics))

Test Score: 0.24

Avatar Execution Metrics Report
Execution Time: 115.02 seconds
Total API Calls: 60
Total Tokens: 110,098 (2,194 in, 107,904 out)
Estimated Cost: $1.0845

Average Time per Call: 1.92 seconds

Tool Usage Breakdown:
-------------------
main_llm: 60 calls



## Optimization

For the optimization of the `Actor` we'll be using `AvatarOptimizer`. It's a DSPy implementation of the [Avatar](https://github.com/zou-group/avatar/) method that optimizes the `Actor` for the given `tools` using a comparator module that optimizes Actor instruction. Note, that Actor is the Module that directs tool execution and flow, it's not the signature that we are passing. It doesn't optimize the instruction of the signature we pass. It takes the following parameters:

* `metric`: Metric that we'll be optimizing for
* `max_iters`: Maximum number of iterations for the optimizer
* `lower_bound`: Lower bound for the metric to classify example as negative
* `upper_bound`: Upper bound for the metric to classify example as positive
* `max_positive_inputs`: Maximum number of positive inputs sampled for comparator
* `max_negative_inputs`: Maximum number of negative inputs sampled for comparator
* `optimize_for`: Whether we want to maximize the metric or minimize it during optimization

Once the optimizer is done we can get the optimized actor and use it for the evaluation.

In [20]:
from new_optimizer import AvatarOptimizerWithMetrics

iterative_monkey = AvatarOptimizerWithMetrics(
    metric=metric,
    max_iters=3,
    max_negative_inputs=10,
    max_positive_inputs=10,
    lower_bound=0.5,
    upper_bound=0.5
)

In [ ]:
result = iterative_monkey.compile(
    student=actor_agent,
    trainset=toolqa_train
)

Processing examples:   0%|          | 0/40 [00:00<?, ?it/s]

The HD-CNN method achieves a reduction in top-1 error by 2.65%, 3.1%, and 1.1% on the CIFAR-100 dataset, but the exact percentage error score is not explicitly mentioned in the retrieved document. | 32.62 => 0.0
The corresponding Bounding_Box_AP score of the CornerNet method on the COCO dataset for the Object Detection task is 42.2% AP. | 42.1 => 0.0
The corresponding accuracy score of the FaceNet method on the YouTube_Faces_DB dataset for the Face_Verification task is 95.12%. | 95.12% => 1.0
The corresponding F1_Full score of the Transition-based improved aligner ensemble method on the LDC2014T12 dataset for the AMR Parsing task is 68.4 Smatch F1 score. | 0.68 => 0.0
The Percentage_error score of the Stochastic_Pooling method on the SVHN dataset for the Image_Classification task is approximately 1% relative test error decrease. | 2.8 => 0.0
The corresponding Model_Entropy score of the PixelRNN method on the CIFAR-10 dataset for the Image_Generation task is not readily available in the

Processing examples:   2%|▎         | 1/40 [01:04<42:05, 64.75s/it]

The Medium_Human-Normalized_Score score of the Ape-X method on the Atari-57 dataset for the Atari_Games task is not directly available from the retrieved sources. Further specific research or access to detailed experimental results from the original Ape-X method publication may be required to obtain this information. | 434.1% => 0.0


Processing examples:  20%|██        | 8/40 [01:06<03:13,  6.05s/it]

...

Processing examples:  20%|██        | 8/40 [00:45<02:20,  4.40s/it]

Unable to find the specific PSNR score for the FAFR_ method on the BSD100 4x upscaling Image Super-Resolution task. Further detailed search or a specific paper reference may be needed. | 26.91 => 0.0
The Recall_50 score for the Mult-DAE method on the Netflix dataset for the Collaborative Filtering task is not readily available from the current search results. Further detailed research or access to specific academic papers or datasets might be required to obtain this specific metric. | 0.438 => 0.0


Processing examples: 100%|██████████| 40/40 [01:31<00:00,  2.28s/it]

The specific Time__ms_ score for the DeepLab method on the Cityscapes dataset for Real-Time Semantic Segmentation is not readily available from the current search results. It is recommended to refer to the original research papers or official documentation of the DeepLab method for precise performance metrics. | 4000 => 0.0
Average Score: 0.4875


Generated new instruction: New Instruction: You will be given `Tools`, which is a list of resources available to accomplish the `Goal`. Your task is to decide which tool to use and what input values to provide based on the user query. To enhance the effectiveness of your actions, ensure that you select the most appropriate tool for the type of information being sought. For instance, use `ARXIV_SEARCH` for specific academic paper-related queries and `WEB_SEARCH` for broader information gathering. This strategic selection will help in obtaining more relevant and precise results. When structuring your queries, aim for specificity and detail. Well-structured queries lead to more precise and relevant outcomes. Avoid vague or general queries, as they can result in incomplete or irrelevant information. For example, include specific dataset names, methods, and metrics in your queries to improve the relevance of the results.

Incorporate a feedback loop into your process, where the results from

In [ ]:
optimized_actor_agent = result["agent"]
optimization_metrics = result["metrics"]

# Now you can process the metrics
print(f"Total optimization cost: ${optimization_metrics['total_cost']:.4f}")
print(f"Final score achieved: {optimization_metrics['final_score']:.3f}")

# Analyze per-iteration performance
for iteration in optimization_metrics['iteration_details']:
    print(f"\nIteration {iteration['iteration']}:")
    print(f"Score: {iteration['score']:.3f}")
    print(f"Comparator tokens in: {iteration['comparator_metrics']['tokens_in']}")
    print(f"Comparator tokens out: {iteration['comparator_metrics']['tokens_out']}")
    print(f"Feedback tokens in: {iteration['feedback_metrics']['tokens_in']}")
    print(f"Feedback tokens out: {iteration['feedback_metrics']['tokens_out']}")
    print(f"Execution time: {iteration['total_iteration_time']:.2f}s")

Total optimization cost: $4.3449
Final score achieved: 0.500

Iteration 0:
Score: 0.300
Comparator tokens in: 34327
Comparator tokens out: 448
Feedback tokens in: 593
Feedback tokens out: 266
Execution time: 85.27s

Iteration 1:
Score: 0.500
Comparator tokens in: 55641
Comparator tokens out: 434
Feedback tokens in: 729
Feedback tokens out: 381
Execution time: 196.46s

Iteration 2:
Score: 0.487
Comparator tokens in: 47721
Comparator tokens out: 469
Feedback tokens in: 879
Feedback tokens out: 472
Execution time: 112.59s


Now we can evaluate our actor module, for this we've provided an implementation of thread safe evaluator that we above as part of class method of `AvatarOptimizer`.

In [23]:
# iterative_monkey.thread_safe_evaluator(toolqa_test, optimized_actor_agent)
batch_num = 4
iterative_monkey.thread_safe_evaluator_batch(toolqa_test, optimized_actor_agent,batch_num)

Processing batch 1 of 4...


Processing examples:   2%|▏         | 1/60 [00:09<09:21,  9.51s/it]

The corresponding Accuracy score of the BB8 method on the LineMOD dataset for the 6D Pose Estimation task is 89.3%. | 83.9% => 0.0
The accuracy score of the Planetoid method on the Cora dataset for the Document Classification task is 75.7%. | 75.7% => 1.0
The corresponding MAP score of the subCNN method on the PASCAL VOC 2007 dataset for the Object Detection task is 68.5%. | 68.5% => 1.0
The Validation perplexity score of the AWD-LSTM-MoS + Partial Shuffle method on the Penn Treebank Word Level dataset for Language Modelling is 53.92. | 55.89 => 0.0
The corresponding Average score of the Multi-task tri-training method on the Multi-Domain Sentiment Dataset for the Sentiment Analysis task is 79.15. | 79.15 => 1.0
The Mean IoU score of the ResNet-38_MS_COCO method on the PASCAL VOC 2012 dataset for the Semantic Segmentation task is 84.9%. | 84.9% => 1.0
The corresponding Percentage error score of the Microsoft 2016b method on the Switchboard + Hub500 dataset for the Speech Recognition tas

Processing examples:   3%|▎         | 2/60 [00:14<06:50,  7.07s/it]

The BLEU score of the ConvS2S method on the IWSLT2015 English-German dataset for the Machine Translation task is 26.73. | 26.73 => 1.0
The Train_Accuracy score of the 450D_DR-BiLSTM_Ensemble method on the SNLI dataset for the Natural Language Inference task is 94.8%. | 94.8 => 1.0
The BLEU score for the Enc-Dec_Att__char_ method on the WMT2015 English-German dataset for the Machine Translation task is 23.5. | 23.45 => 0.5
The corresponding Frame fps score of the CRF-RNN method on the Cityscapes dataset for Real-Time Semantic Segmentation is 1.4 fps. | 1.4 => 1.0
The corresponding Rank-1 score of the PDF method on the Market-1501 dataset for the Person Re-Identification task is 83.58%. | 84.14 => 0.0
The corresponding Accuracy score of the MAML method on the OMNIGLOT 1-Shot Learning dataset for the Few-Shot Image Classification task is 98.7%. | 98.7% => 1.0
The corresponding Accuracy score of the DeepId2 method on the Labeled Faces in the Wild dataset for the Face Verification task is 9

...

Processing examples:  62%|██████▏   | 37/60 [01:16<00:28,  1.25s/it]

The Hits@1 score for the ComplEx method on the WN18 dataset for the Link Prediction task is not readily available from the sources searched. It is recommended to refer to the original paper "Complex Embeddings for Simple Link Prediction" by Théo Trouillon et al. for the specific value, as the search did not yield the exact score. | 0.936 => 0.0
The Arc-hybrid method achieves a UAS score of 90.22 on the Penn Treebank for the Dependency Parsing task. | 93.56 => 0.0


Processing examples: 100%|██████████| 60/60 [02:21<00:00,  2.36s/it]

The Checkerboards method achieves a log-average miss rate of 18.5% on the Reasonable subset of the Caltech dataset for the Pedestrian Detection task. | 17.1 => 0.0


0.45